# **Fine-tuning Wav2Vec2 for English ASR with 🤗 Transformers**

Wav2Vec2 is a pretrained model for Automatic Speech Recognition (ASR) and was released in [September 2020](https://ai.facebook.com/blog/wav2vec-20-learning-the-structure-of-speech-from-raw-audio/) by Alexei Baevski, Michael Auli, and Alex Conneau.

Using a novel contrastive pretraining objective, Wav2Vec2 learns powerful speech representations from more than 50.000 hours of unlabeled speech. Similar, to [BERT's masked language modeling](http://jalammar.github.io/illustrated-bert/), the model learns contextualized speech representations by randomly masking feature vectors before passing them to a transformer network.

![wav2vec2_structure](https://raw.githubusercontent.com/patrickvonplaten/scientific_images/master/wav2vec2.png)

For the first time, it has been shown that pretraining, followed by fine-tuning on very little labeled speech data achieves competitive results to state-of-the-art ASR systems. Using as little as 10 minutes of labeled data, Wav2Vec2 yields a word error rate (WER) of less than 5% on the clean test set of [LibriSpeech](https://huggingface.co/datasets/librispeech_asr) - *cf.* with Table 9 of the [paper](https://arxiv.org/pdf/2006.11477.pdf).

In this notebook, we will give an in-detail explanation of how Wav2Vec2's pretrained checkpoints can be fine-tuned on any English ASR dataset. Note that in this notebook, we will fine-tune Wav2Vec2 without making use of a language model. It is much simpler to use Wav2Vec2 without a language model as an end-to-end ASR system and it has been shown that a standalone Wav2Vec2 acoustic model achieves impressive results. For demonstration purposes, we fine-tune the "base"-sized [pretrained checkpoint](https://huggingface.co/facebook/wav2vec2-base) on the rather small [Timit](https://huggingface.co/datasets/timit_asr) dataset that contains just 5h of training data.

Wav2Vec2 is fine-tuned using Connectionist Temporal Classification (CTC), which is an algorithm that is used to train neural networks for sequence-to-sequence problems and mainly in Automatic Speech Recognition and handwriting recognition. 

I highly recommend reading the blog post [Sequence Modeling with CTC (2017)](https://distill.pub/2017/ctc/) very well-written blog post by Awni Hannun.

First, let's try to get a good GPU in our colab! With Google Colab's free version it's sadly becoming much harder to get access to a good GPU. With Google Colab Pro, one has a much easier time getting access to a V100 or P100 GPU however.

In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Thu Oct 27 10:40:02 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

Before we start, let's install both `datasets` and `transformers` from master. Also, we need the `librosa` package to load audio files and the `jiwer` to evaluate our fine-tuned model using the [word error rate (WER)](https://huggingface.co/metrics/wer) metric ${}^1$.

In [2]:
%%capture
!pip install datasets==1.18.3
!pip install transformers==4.17.0
!pip install jiwer

Next we strongly suggest to upload your training checkpoints directly to the [🤗 Hub](https://huggingface.co/) while training. The [🤗 Hub](https://huggingface.co/) has integrated version control so you can be sure that no model checkpoint is getting lost during training. 

To do so you have to store your authentication token from the Hugging Face website (sign up [here](https://huggingface.co/join) if you haven't already!)

In [ ]:
from huggingface_hub import notebook_login

notebook_login()


Then you need to install Git-LFS to upload your model checkpoints:

In [ ]:
%%capture
!apt install git-lfs




---

${}^1$ Timit is usually evaluated using the phoneme error rate (PER), but by far the most common metric in ASR is the word error rate (WER). To keep this notebook as general as possible we decided to evaluate the model using WER.

## Prepare Data, Tokenizer, Feature Extractor

ASR models transcribe speech to text, which means that we both need a feature extractor that processes the speech signal to the model's input format, *e.g.* a feature vector, and a tokenizer that processes the model's output format to text. 

In 🤗 Transformers, the Wav2Vec2 model is thus accompanied by both a tokenizer, called [Wav2Vec2CTCTokenizer](https://huggingface.co/transformers/master/model_doc/wav2vec2.html#wav2vec2ctctokenizer), and a feature extractor, called [Wav2Vec2FeatureExtractor](https://huggingface.co/transformers/master/model_doc/wav2vec2.html#wav2vec2featureextractor).

Let's start by creating the tokenizer responsible for decoding the model's predictions.

### Create Wav2Vec2CTCTokenizer

The [pretrained Wav2Vec2 checkpoint]( ) maps the speech signal to a sequence of context representations as illustrated in the figure above. A fine-tuned Wav2Vec2 checkpoint needs to map this sequence of context representations to its corresponding transcription so that a linear layer has to be added on top of the transformer block (shown in yellow). This linear layer is used to classifies each context representation to a token class analogous how, *e.g.*, after pretraining a linear layer is added on top of BERT's embeddings for further classification - *cf.* with *"BERT"* section of this [blog post](https://huggingface.co/blog/warm-starting-encoder-decoder).

The output size of this layer corresponds to the number of tokens in the vocabulary, which does **not** depend on Wav2Vec2's pretraining task, but only on the labeled dataset used for fine-tuning. So in the first step, we will take a look at Timit and define a vocabulary based on the dataset's transcriptions.

Let's start by loading the dataset and taking a look at its structure.

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cd /content
# !mkdir /content/sample_data
# !mkdir /content/sample_data/ATC_data
!wget -P /content/sample_data/ATC_data http://www2.spsc.tugraz.at/databases/ATCOSIM/.ISO/atcosim.iso

--2022-09-13 10:05:41--  http://www2.spsc.tugraz.at/databases/ATCOSIM/.ISO/atcosim.iso
Resolving www2.spsc.tugraz.at (www2.spsc.tugraz.at)... 129.27.140.126
Connecting to www2.spsc.tugraz.at (www2.spsc.tugraz.at)|129.27.140.126|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www2.spsc.tugraz.at//databases/ATCOSIM/.ISO/atcosim.iso [following]
--2022-09-13 10:05:43--  https://www2.spsc.tugraz.at//databases/ATCOSIM/.ISO/atcosim.iso
Connecting to www2.spsc.tugraz.at (www2.spsc.tugraz.at)|129.27.140.126|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2597789696 (2.4G) [application/x-iso9660-image]
Saving to: ‘/content/sample_data/ATC_data/atcosim.iso’

atcosim.iso         100%[===================>]   2.42G  13.6MB/s    in 3m 4s   

2022-09-13 10:08:48 (13.4 MB/s) - ‘/content/sample_data/ATC_data/atcosim.iso’ saved [2597789696/2597789696]



In [ ]:
%cd sample_data/ATC_data
!ls
# !7z x atcosim.ISO

/content/sample_data/ATC_data
atcosim.iso


In [ ]:
!7z x atcosim.iso


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.00GHz (50653),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan         1 file, 2597789696 bytes (2478 MiB)

Extracting archive: atcosim.iso
  1% 100 Open             --
Path = atcosim.iso
Type = Udf
Physical Size = 2597789696
Comment = ATCOSIM
Cluster Size = 2048
Created = 2008-05-26 09:41:25

102%      4% 496 - WAVdata/sm1/sm1_04/sm1_04_004.wav                                              9% 989         11% 1180 - WAVdata/sm2/sm2_01/sm2_01_003.wav                                              12% 1316 - WAVdata/sm2/sm2_01/sm2_01_139.wav

In [3]:
from datasets import load_dataset, load_metric
import pandas as pd
atcosim_path = '/content/drive/MyDrive/Colab Notebooks/full_data3.csv'

atcosim_train = pd.read_csv(atcosim_path, encoding='utf-8', encoding_errors='strict')
df = pd.DataFrame(atcosim_train)
print(df)

      directory subdirectory    filename speaker_id  session_id  utterance_id  \
0           sm1       sm1_01  sm1_01_001        sm1           1             1   
1           sm1       sm1_01  sm1_01_002        sm1           1             2   
2           sm1       sm1_01  sm1_01_003        sm1           1             3   
3           sm1       sm1_01  sm1_01_004        sm1           1             4   
4           sm1       sm1_01  sm1_01_005        sm1           1             5   
...         ...          ...         ...        ...         ...           ...   
10073       sm2       sm2_09  sm2_09_204        sm2           9           204   
10074       sm2       sm2_09  sm2_09_205        sm2           9           205   
10075       sm2       sm2_09  sm2_09_206        sm2           9           206   
10076       sm2       sm2_09  sm2_09_207        sm2           9           207   
10077       sm2       sm2_09  sm2_09_208        sm2           9           208   

                           

### Adding path column to the CSV file:

In [4]:
import csv
import numpy as np

# ######### local dataset
# prefix_path = "/content/drive/MyDrive/ATCOSIM dataset/WAVdata/"

# ########## dataset uploaded in goolge drive
# data_path = "/content/sample_data/ATC_data/WAVdata/"
data_path = "/content/drive/MyDrive/ATCOSIM dataset/WAVdata/"
# data_path = "/home/guest/atcasr/Downloads/ATC_data/WAVdata/"

df = pd.DataFrame(atcosim_train)
rows_len = len(df)
final_len = 3000

col1 = [None] * rows_len
col2 = [None] * rows_len

new_atcosim_path = "/content/drive/MyDrive/Colab Notebooks/new_fulldata22.csv"

for pc in range(rows_len):
  col1[pc] = data_path

for id, dr_path in enumerate(df["directory"]):
    col1[id] += dr_path
    col1[id] += "/"

for id, dr_path in enumerate(df["subdirectory"]):
    col1[id] += dr_path
    col1[id] += "/"

for id, dr_path in enumerate(df["filename"]):
    col1[id] += dr_path
    col1[id] += ".wav"

for id, trans in enumerate(df["transcription"]):
    col2[id] = trans

new_df = pd.DataFrame({"path" : col1[:final_len], "transcription": col2[:final_len]}, index = (list(range(final_len))))
print(new_df)
new_df.to_csv(new_atcosim_path)


                                                   path  \
0     /content/drive/MyDrive/ATCOSIM dataset/WAVdata...   
1     /content/drive/MyDrive/ATCOSIM dataset/WAVdata...   
2     /content/drive/MyDrive/ATCOSIM dataset/WAVdata...   
3     /content/drive/MyDrive/ATCOSIM dataset/WAVdata...   
4     /content/drive/MyDrive/ATCOSIM dataset/WAVdata...   
...                                                 ...   
2995  /content/drive/MyDrive/ATCOSIM dataset/WAVdata...   
2996  /content/drive/MyDrive/ATCOSIM dataset/WAVdata...   
2997  /content/drive/MyDrive/ATCOSIM dataset/WAVdata...   
2998  /content/drive/MyDrive/ATCOSIM dataset/WAVdata...   
2999  /content/drive/MyDrive/ATCOSIM dataset/WAVdata...   

                                          transcription  
0     ~p ~s ~a eight one zero turn right to trasadin...  
1     lufthansa five three one eight contact zurich ...  
2     ~p ~s ~a eight one zero contact zurich one thr...  
3               sabena four eight one rhein identified   
4

In [5]:
print(atcosim_train)

      directory subdirectory    filename speaker_id  session_id  utterance_id  \
0           sm1       sm1_01  sm1_01_001        sm1           1             1   
1           sm1       sm1_01  sm1_01_002        sm1           1             2   
2           sm1       sm1_01  sm1_01_003        sm1           1             3   
3           sm1       sm1_01  sm1_01_004        sm1           1             4   
4           sm1       sm1_01  sm1_01_005        sm1           1             5   
...         ...          ...         ...        ...         ...           ...   
10073       sm2       sm2_09  sm2_09_204        sm2           9           204   
10074       sm2       sm2_09  sm2_09_205        sm2           9           205   
10075       sm2       sm2_09  sm2_09_206        sm2           9           206   
10076       sm2       sm2_09  sm2_09_207        sm2           9           207   
10077       sm2       sm2_09  sm2_09_208        sm2           9           208   

                           

In [ ]:
prefix_path = '/content/drive/MyDrive/Colab Notebooks/'
new_atcosim_path = prefix_path +'final_data_edited.csv'


### Load Train, Test and validation dataset

In [ ]:
from datasets import load_dataset, load_metric, Audio 
import pandas as pd

train_size = 8000
atcosim_csv = pd.read_csv(new_atcosim_path)

df = atcosim_csv[0 : train_size]
df.to_csv(prefix_path + "train_set.csv", index=False)

df = atcosim_csv[train_size+1 : ]
df.to_csv(prefix_path + "test_set.csv", index=False)

atcosim_dataset = load_dataset("csv", 
                             data_files={'train': prefix_path + "train_set.csv",
                                         'test': prefix_path + "test_set.csv"}, delimiter = ",")

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-ad9ea3b1a8362f37/0.0.0/6b9057d9e23d9d8a2f05b985917a0da84d70c5dae3d22ddd8a3f22fb01c69d9e. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

### separating the *dataset* to train and test set:


In [6]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
import pandas as pd

data = pd.read_csv(new_atcosim_path)

df_train, df_test = train_test_split(data, test_size = 0.01)
# test, train = split_data(data)

print(df_train)
print(df_test)

      Unnamed: 0                                               path  \
350          350  /content/drive/MyDrive/ATCOSIM dataset/WAVdata...   
478          478  /content/drive/MyDrive/ATCOSIM dataset/WAVdata...   
527          527  /content/drive/MyDrive/ATCOSIM dataset/WAVdata...   
2717        2717  /content/drive/MyDrive/ATCOSIM dataset/WAVdata...   
2613        2613  /content/drive/MyDrive/ATCOSIM dataset/WAVdata...   
...          ...                                                ...   
691          691  /content/drive/MyDrive/ATCOSIM dataset/WAVdata...   
2253        2253  /content/drive/MyDrive/ATCOSIM dataset/WAVdata...   
1971        1971  /content/drive/MyDrive/ATCOSIM dataset/WAVdata...   
2589        2589  /content/drive/MyDrive/ATCOSIM dataset/WAVdata...   
244          244  /content/drive/MyDrive/ATCOSIM dataset/WAVdata...   

                                          transcription  
350   alitalia one three five contact zurich one thr...  
478   alitalia four zero one no

In [7]:
df_train = pd.DataFrame(df_train)
df_test = pd.DataFrame(df_test)

In [8]:
df_train
# df_test

,Unnamed: 0,path,transcription
350,350,/content/drive/MyDrive/ATCOSIM dataset/WAVdata...,alitalia one three five contact zurich one thr...
478,478,/content/drive/MyDrive/ATCOSIM dataset/WAVdata...,alitalia four zero one now continue climb to l...
527,527,/content/drive/MyDrive/ATCOSIM dataset/WAVdata...,alitalia one four nine for shortcut right one ...
2717,2717,/content/drive/MyDrive/ATCOSIM dataset/WAVdata...,sabena eight zero one contact zurich on one th...
2613,2613,/content/drive/MyDrive/ATCOSIM dataset/WAVdata...,hapag lloyd six five three call zurich on one ...
...,...,...,...
691,691,/content/drive/MyDrive/ATCOSIM dataset/WAVdata...,gulf air zero zero five contact rhein radar on...
2253,2253,/content/drive/MyDrive/ATCOSIM dataset/WAVdata...,merair six nine five two climb to flight level...
1971,1971,/content/drive/MyDrive/ATCOSIM dataset/WAVdata...,alitalia six four seven four good afternoon id...
2589,2589,/content/drive/MyDrive/ATCOSIM dataset/WAVdata...,malaysian two radar contact


In [9]:
import os

abs_path_to_data = os.path.join("/content", "sample_data", "ATC_data", "train_test_data", "")
!mkdir {abs_path_to_data}
print(abs_path_to_data)
save_path = "/".join(abs_path_to_data.split('/'))
print(save_path)

print(len(df_test))
print(len(df_train))

df_train.to_csv(f"{save_path}/train.csv", sep="\t", encoding="utf-8", index=False)
df_test.to_csv(f"{save_path}/test.csv", sep="\t", encoding="utf-8", index=False)
!ls {save_path}


mkdir: cannot create directory ‘/content/sample_data/ATC_data/train_test_data/’: File exists
/content/sample_data/ATC_data/train_test_data/
/content/sample_data/ATC_data/train_test_data/
30
2970
test.csv  train.csv


In [10]:
from datasets import load_dataset, load_metric, Audio 

atcosim_train = load_dataset("csv", data_files={"train": abs_path_to_data+"train.csv"}, delimiter="\t")["train"]
atcosim_test = load_dataset("csv", data_files={"test": abs_path_to_data+"test.csv"}, delimiter="\t")["test"]

print(atcosim_train)
print(atcosim_test)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-36c0e0c6f9af3aeb/0.0.0/6b9057d9e23d9d8a2f05b985917a0da84d70c5dae3d22ddd8a3f22fb01c69d9e. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-348974e6f63cdae7/0.0.0/6b9057d9e23d9d8a2f05b985917a0da84d70c5dae3d22ddd8a3f22fb01c69d9e. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

Dataset({
    features: ['Unnamed: 0', 'path', 'transcription'],
    num_rows: 2970
})
Dataset({
    features: ['Unnamed: 0', 'path', 'transcription'],
    num_rows: 30
})


In [72]:
# def prepare_columns(batch):
#   batch["path"] = batch[1]
#   batch["transcription"] = batch[2]
#   return batch

In [92]:
# atcosim_train = atcosim_train.map(prepare_columns, remove_columns=atcosim_train.column_names)
# atcosim_test = atcosim_test.map(prepare_columns, remove_columns=atcosim_test.column_names)

Many ASR datasets only provide the target text, `'text'` for each audio `'audio'` and file `'file'`. Timit actually provides much more information about each audio file, such as the `'phonetic_detail'`, etc., which is why many researchers choose to evaluate their models on phoneme classification instead of speech recognition when working with Timit. However, we want to keep the notebook as general as possible, so that we will only consider the transcribed text for fine-tuning.



Let's write a short function to display some random samples of the dataset and run it a couple of times to get a feeling for the transcriptions.

In [11]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

In [12]:
# show_random_elements(atcosim["train"].remove_columns(["audio", "file"]), num_examples=10)
# atcosim['train']
show_random_elements(atcosim_train.remove_columns(["path"]), num_examples=10)

,Unnamed: 0,transcription
0,940,bafair four correction belgian airforce four four report your heading
1,1394,algerie two zero seven three contact zurich one three four decimal six
2,987,[EMPTY]
3,2634,alitalia four eight five radar contact
4,780,india mike lima lima guten tag identified
5,97,alitalia station co= ah alitalia four eight seven i got it you're identified
6,2964,fox
7,2239,bee med one proceed now own navigation to lupen
8,1819,ah air malta zero zero four you're in radar contact
9,2449,lufthansa five five zero four inbound to trasadingen


Alright! The transcriptions look very clean and the language seems to correspond more to written text than dialogue. This makes sense taking into account that [ATCOSIM](https://www2.spsc.tugraz.at/databases/ATCOSIM/WAVdata/) is a read speech corpus.

We can see that the transcriptions contain some special characters, such as `,.?!;:`. Without a language model, it is much harder to classify speech chunks to such special characters because they don't really correspond to a characteristic sound unit. *E.g.*, the letter `"s"` has a more or less clear sound, whereas the special character `"."` does not.
Also in order to understand the meaning of a speech signal, it is usually not necessary to include special characters in the transcription.

In addition, we normalize the text to only have lower case letters and append a word separator token at the end.

In [13]:
import re
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\]\[\=\<\>\@\/]'

def remove_special_characters(batch):     
    batch["transcription"] = re.sub(chars_to_ignore_regex, '', batch["transcription"]).lower() + " "
    return batch

In [14]:
atcosim_train = atcosim_train.map(remove_special_characters)
atcosim_test = atcosim_test.map(remove_special_characters)


0ex [00:00, ?ex/s]

0ex [00:00, ?ex/s]

In [15]:
show_random_elements(atcosim_train.remove_columns(["path"]), num_examples=20)

,Unnamed: 0,transcription
0,2941,lufthansa eight two six eight confirm the rate of climb please
1,2378,swissair eight zero six contact rhein one two seven three seven tschuss
2,1762,lufthansa four five eight two climb flight level two nine zero
3,1716,meridiana co correction merair six nine four four ah climb to flight level three three zero rate of climb one thousand five hundred or more
4,142,alitalia two nine two turn left to gotil
5,2500,roger turn right heading one nine zero
6,515,britannia two two six alfa radar con correction two three six alfa radar contact
7,706,foxtrot sierra india contact zurich one three four decimal six
8,548,hapag lloyd one three seven guten tag radar contact
9,2146,lufthansa triple three zero report your mach number


Good! This looks better. We have removed most special characters from transcriptions and normalized them to lower-case only.

In CTC, it is common to classify speech chunks into letters, so we will do the same here. 
Let's extract all distinct letters of the training and test data and build our vocabulary from this set of letters.

We write a mapping function that concatenates all transcriptions into one long transcription and then transforms the string into a set of chars. 
It is important to pass the argument `batched=True` to the `map(...)` function so that the mapping function has access to all transcriptions at once.

In [16]:
def extract_all_chars(batch):
  all_text = " ".join(batch["transcription"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}

In [17]:
vocabs_train = atcosim_train.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=atcosim_train.column_names)
vocabs_test = atcosim_test.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=atcosim_test.column_names)


  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

Now, we create the union of all distinct letters in the training dataset and test dataset and convert the resulting list into an enumerated dictionary.

In [18]:
vocab_list = list(set(vocabs_train["vocab"][0]) | set(vocabs_test["vocab"][0]))

In [19]:
vocab_dict = {v: k for k, v in enumerate(vocab_list)}
vocab_dict 

{' ': 0,
 'c': 1,
 'b': 2,
 'e': 3,
 's': 4,
 "'": 5,
 'o': 6,
 't': 7,
 'h': 8,
 'm': 9,
 'j': 10,
 'z': 11,
 'v': 12,
 'p': 13,
 'i': 14,
 'y': 15,
 'g': 16,
 'q': 17,
 'r': 18,
 'l': 19,
 'k': 20,
 'd': 21,
 'u': 22,
 'f': 23,
 '~': 24,
 'w': 25,
 'a': 26,
 'n': 27,
 'x': 28}

Cool, we see that all letters of the alphabet occur in the dataset (which is not really surprising) and we also extracted the special characters `" "` and `'`. Note that we did not exclude those special characters because: 

- The model has to learn to predict when a word finished or else the model prediction would always be a sequence of chars which would make it impossible to separate words from each other.
- In English, we need to keep the `'` character to differentiate between words, *e.g.*, `"it's"` and `"its"` which have very different meanings.

To make it clearer that `" "` has its own token class, we give it a more visible character `|`. In addition, we also add an "unknown" token so that the model can later deal with characters not encountered in Timit's training set. 

Finally, we also add a padding token that corresponds to CTC's "*blank token*". The "blank token" is a core component of the CTC algorithm. For more information, please take a look at the "Alignment" section [here](https://distill.pub/2017/ctc/).

In [20]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]

In [21]:
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
len(vocab_dict)
vocab_dict

{'c': 1,
 'b': 2,
 'e': 3,
 's': 4,
 "'": 5,
 'o': 6,
 't': 7,
 'h': 8,
 'm': 9,
 'j': 10,
 'z': 11,
 'v': 12,
 'p': 13,
 'i': 14,
 'y': 15,
 'g': 16,
 'q': 17,
 'r': 18,
 'l': 19,
 'k': 20,
 'd': 21,
 'u': 22,
 'f': 23,
 '~': 24,
 'w': 25,
 'a': 26,
 'n': 27,
 'x': 28,
 '|': 0,
 '[UNK]': 29,
 '[PAD]': 30}

Cool, now our vocabulary is complete and consists of 38 tokens, which means that the linear layer that we will add on top of the pretrained Wav2Vec2 checkpoint will have an output dimension of 38.

Let's now save the vocabulary as a json file.

In [22]:
import json
with open('/content/sample_data/ATC_data/vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

In [23]:
save_dir = "/content/drive/MyDrive/wav2vec2-robust-atcosim"
# model_name_or_path = "m3hrdadfi/wav2vec2-large-xlsr-persian-v2"

In [38]:
# from transformers.trainer_utils import get_last_checkpoint

# last_checkpoint = None
# if os.path.exists(save_dir):
#     last_checkpoint = get_last_checkpoint(save_dir)

# print(last_checkpoint if last_checkpoint else str(None))

In a final step, we use the json file to instantiate an object of the `Wav2Vec2CTCTokenizer` class.

In [24]:
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer("/content/sample_data/ATC_data/vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

In [25]:
text = "Wish you a safe flight"
print(" ".join(tokenizer.tokenize(text)))
print(tokenizer.decode(tokenizer.encode(text)))

W i s h | y o u | a | s a f e | f l i g h t
[UNK]ish you a safe flight


If one wants to re-use the just created tokenizer with the fine-tuned model of this notebook, it is strongly advised to upload the `tokenizer` to the [🤗 Hub](https://huggingface.co/). Let's call the repo to which we will upload the files
`"wav2vec2-base-timit-demo-colab"`:

In [26]:
repo_name = "wav2vec2-robut-atcosim"

and upload the tokenizer to the [🤗 Hub](https://huggingface.co/).

In [ ]:
!pip install git-lfs


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# tokenizer.push_to_hub(repo_name)

Great, you can see the just created repository under `https://huggingface.co/<your-username>/wav2vec2-base-timit-demo-colab`

### Create Wav2Vec2 Feature Extractor

Speech is a continuous signal and to be treated by computers, it first has to be discretized, which is usually called **sampling**. The sampling rate hereby plays an important role in that it defines how many data points of the speech signal are measured per second. Therefore, sampling with a higher sampling rate results in a better approximation of the *real* speech signal but also necessitates more values per second.

A pretrained checkpoint expects its input data to have been sampled more or less from the same distribution as the data it was trained on. The same speech signals sampled at two different rates have a very different distribution, *e.g.*, doubling the sampling rate results in data points being twice as long. Thus, 
before fine-tuning a pretrained checkpoint of an ASR model, it is crucial to verify that the sampling rate of the data that was used to pretrain the model matches the sampling rate of the dataset used to fine-tune the model.

Wav2Vec2 was pretrained on the audio data of [LibriSpeech](https://huggingface.co/datasets/librispeech_asr) and LibriVox which both were sampling with 16kHz. Our fine-tuning dataset, [Timit](hhtps://huggingface.co/datasets/timit_asr), was luckily also sampled with 16kHz. If the fine-tuning dataset would have been sampled with a rate lower or higher than 16kHz, we first would have had to up or downsample the speech signal to match the sampling rate of the data used for pretraining. 



A Wav2Vec2 feature extractor object requires the following parameters to be instantiated:

- `feature_size`: Speech models take a sequence of feature vectors as an input. While the length of this sequence obviously varies, the feature size should not. In the case of Wav2Vec2, the feature size is 1 because the model was trained on the raw speech signal ${}^2$.
- `sampling_rate`: The sampling rate at which the model is trained on.
- `padding_value`: For batched inference, shorter inputs need to be padded with a specific value
- `do_normalize`: Whether the input should be *zero-mean-unit-variance* normalized or not. Usually, speech models perform better when normalizing the input
- `return_attention_mask`: Whether the model should make use of an `attention_mask` for batched inference. In general, models should **always** make use of the `attention_mask` to mask padded tokens. However, due to a very specific design choice of `Wav2Vec2`'s "base" checkpoint, better results are achieved when using no `attention_mask`. This is **not** recommended for other speech models. For more information, one can take a look at [this](https://github.com/pytorch/fairseq/issues/3227) issue. **Important** If you want to use this notebook to fine-tune [large-lv60](https://huggingface.co/facebook/wav2vec2-large-lv60), this parameter should be set to `True`.

In [27]:
from transformers import Wav2Vec2FeatureExtractor

# feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=False)
feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=False)


Great, Wav2Vec2's feature extraction pipeline is thereby fully defined!

To make the usage of Wav2Vec2 as user-friendly as possible, the feature extractor and tokenizer are *wrapped* into a single `Wav2Vec2Processor` class so that one only needs a `model` and `processor` object.

In [28]:
from transformers import Wav2Vec2Processor

# processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)
processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)
# print(tokenizer)

In [29]:
if len(processor.tokenizer.get_vocab()) == len(processor.tokenizer):
    print(len(processor.tokenizer))
# print(len(processor.tokenizer.get_vocab()))
# print(len(processor.tokenizer))

31


Next, we can prepare the dataset.

### Preprocess Data

So far, we have not looked at the actual values of the speech signal but just the transcription. In addition to `'text'`, our datasets include two more column names `'file'` and `'audio'`. `'file'` states the absolute path of the audio file. Let's take a look.

`Wav2Vec2` expects the input in the format of a 1-dimensional array of 16 kHz. This means that the audio file has to be loaded and resampled.

 Thankfully, `datasets` does this automatically when calling the column `audio`. Let try it out. 

We can see that the audio file has automatically been loaded. This is thanks to the new [`"Audio"` feature](https://huggingface.co/docs/datasets/package_reference/main_classes.html?highlight=audio#datasets.Audio) introduced in `datasets == 4.13.3`, which loads and resamples audio files on-the-fly upon calling.

The sampling rate is set to 16kHz which is what `Wav2Vec2` expects as an input.

Great, let's listen to a couple of audio files to better understand the dataset and verify that the audio was correctly loaded. 

**Note**: *You can click the following cell a couple of times to listen to different speech samples.*

In [ ]:
# import IPython.display as ipd
# import librosa as rosa
# import numpy as np
# import random

# rand_int = random.randint(0, len(atcosim_dataset["train"]))

# print(atcosim_dataset["train"][rand_int]["transcription"])
# audio_rate, sample_rate = rosa.load(atcosim_dataset["train"][rand_int]["path"], sr = None)
# # ipd.Audio(data=np.asarray(atcosim_dataset["train"][rand_int]["path"]), autoplay=True, rate=16000)
# ipd.Audio(audio_rate, autoplay=True, rate=sample_rate)
# # print(sample_rate)

lufthansa four four one six direct to trasadingen  


Great, now we've successfully read in all the audio files, but since we know that ATCOSIM is sampled at 32kHz, we need to resample the audio files to 16kHz. 


In [30]:
import torchaudio
import librosa
import numpy as np

target_sampling_rate = 16_000

def speech_file_to_array_fn(batch):
    speech_array, sampling_rate = torchaudio.load(batch["path"])
    speech_array = speech_array.squeeze().numpy()
    speech_array = librosa.resample(np.asarray(speech_array), sampling_rate, target_sampling_rate)
    
    
    batch["speech"] = speech_array
    batch["sampling_rate"] = target_sampling_rate
    batch["duration_in_seconds"] = len(batch["speech"]) / target_sampling_rate
    batch["target_text"] = batch["transcription"]
    return batch

In [31]:
atcosim_train = atcosim_train.map(speech_file_to_array_fn, remove_columns=atcosim_train.column_names, num_proc=4)
atcosim_test = atcosim_test.map(speech_file_to_array_fn, remove_columns=atcosim_test.column_names, num_proc=4)

In [32]:
atcosim_train

Dataset({
    features: ['speech', 'sampling_rate', 'duration_in_seconds', 'target_text'],
    num_rows: 2970
})

In [33]:
atcosim_test

Dataset({
    features: ['speech', 'sampling_rate', 'duration_in_seconds', 'target_text'],
    num_rows: 30
})

In [34]:
atcosim_train[0]["sampling_rate"]

16000

It can be heard, that the speakers change along with their speaking rate, accent, etc. Overall, the recordings sound relatively clear though, which is to be expected from a read speech corpus.

Let's do a final check that the data is correctly prepared, by printing the shape of the speech input, its transcription, and the corresponding sampling rate.

**Note**: *You can click the following cell a couple of times to verify multiple samples.*

In [35]:
import IPython.display as ipd
import numpy as np
import random

sample = atcosim_train
rand_int = random.randint(0, len(sample))

print("Target text:", sample[rand_int]["target_text"])
print("Input array shape:", np.asarray(sample[rand_int]["speech"]).shape)
print("Sampling rate:", sample[rand_int]["sampling_rate"])

ipd.Audio(data=np.asarray(sample[rand_int]["speech"]), autoplay=True, rate=16000)

Target text: alitalia two nine one contact zurich one three four decimal six  
Input array shape: (82215,)
Sampling rate: 16000


In [127]:
min_duration_in_seconds = 5.0
max_duration_in_seconds = 10.0

def filter_by_max_duration(batch):
   return min_duration_in_seconds <= batch["duration_in_seconds"] <= max_duration_in_seconds

In [129]:
print(f"Split sizes [BEFORE]: {len(atcosim_train)} train and {len(atcosim_test)} validation.")

_atcosim_train = atcosim_train.filter(filter_by_max_duration, num_proc=24)
#_common_voice_test = common_voice_test
_atcosim_test = atcosim_test.filter(filter_by_max_duration, num_proc=4)

print(f"Split sizes [AFTER]: {len(_atcosim_train)} train and {len(_atcosim_test)} validation.")

Split sizes [BEFORE]: 2970 train and 30 validation.
Split sizes [AFTER]: 506 train and 4 validation.


In [36]:
import IPython.display as ipd
import numpy as np
import random

sample = atcosim_train
rand_int = random.randint(0, len(sample))

print("Target text:", sample[rand_int]["target_text"])
print("Input array shape:", np.asarray(sample[rand_int]["speech"]).shape)
print("Sampling rate:", sample[rand_int]["sampling_rate"])

ipd.Audio(data=np.asarray(sample[rand_int]["speech"]), autoplay=True, rate=16000)

Target text: lufthansa four two zero seven contact rhein radar one three two decimal four  
Input array shape: (97140,)
Sampling rate: 16000


Good! Everything looks fine - the data is a 1-dimensional array, the sampling rate always corresponds to 32kHz, and the target text is normalized.

Finally, we can process the dataset to the format expected by the model for training. We will make use of the `map(...)` function.

First, we load and resample the audio data, simply by calling `batch["audio"]`.
Second, we extract the `input_values` from the loaded audio file. In our case, the `Wav2Vec2Processor` only normalizes the data. For other speech models, however, this step can include more complex feature extraction, such as [Log-Mel feature extraction](https://en.wikipedia.org/wiki/Mel-frequency_cepstrum).
Third, we encode the transcriptions to label ids.

**Note**: This mapping function is a good example of how the `Wav2Vec2Processor` class should be used. In "normal" context, calling `processor(...)` is redirected to `Wav2Vec2FeatureExtractor`'s call method. When wrapping the processor into the `as_target_processor` context, however, the same method is redirected to `Wav2Vec2CTCTokenizer`'s call method.
For more information please check the [docs](https://huggingface.co/transformers/master/model_doc/wav2vec2.html#transformers.Wav2Vec2Processor.__call__).

In [37]:
def prepare_dataset(batch):
    # check that all files have the correct sampling rate
    assert (
        len(set(batch["sampling_rate"])) == 1
    ), f"Make sure all inputs have the same sampling rate of {processor.feature_extractor.sampling_rate}."
    # print(batch["sampling_rate"][0])
    batch["input_values"] = processor(batch["speech"], sampling_rate=batch["sampling_rate"][0]).input_values
    # print(batch["input_values"])
    with processor.as_target_processor():
        batch["labels"] = processor(batch["target_text"]).input_ids
    return batch

Let's apply the data preparation function to all examples.


In [38]:
_atcosim_train = atcosim_train.map(prepare_dataset, remove_columns = atcosim_train.column_names, batch_size=8, num_proc=16, batched=True)
_atcosim_test = atcosim_test.map(prepare_dataset, remove_columns = atcosim_test.column_names, batch_size=8, num_proc=16, batched=True)

/usr/local/lib/python3.7/dist-packages/transformers/feature_extraction_utils.py:169: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  tensor = as_tensor(value)
/usr/local/lib/python3.7/dist-packages/transformers/feature_extraction_utils.py:169: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  tensor = as_tensor(value)
/usr/local/lib/python3.7/dist-packages/transformers/feature_extraction_utils.py:169: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths o

**Note**: Currently `datasets` make use of [`torchaudio`](https://pytorch.org/audio/stable/index.html) and [`librosa`](https://librosa.org/doc/latest/index.html) for audio loading and resampling. If you wish to implement your own costumized data loading/sampling, feel free to just make use of the `"path"` column instead and disregard the `"audio"` column.

Long input sequences require a lot of memory. Since `Wav2Vec2` is based on `self-attention` the memory requirement scales quadratically with the input length for long input sequences (*cf.* with [this](https://www.reddit.com/r/MachineLearning/comments/genjvb/d_why_is_the_maximum_input_sequence_length_of/) reddit post). For this demo, let's filter all sequences that are longer than 4 seconds out of the training dataset.

In [ ]:
# max_input_length_in_sec = 4.0
# atcosim_train = atcosim_train.filter(lambda x: x < max_input_length_in_sec * processor.feature_extractor.sampling_rate, input_columns=["input_length"])

Awesome, now we are ready to start training!

## Training & Evaluation

The data is processed so that we are ready to start setting up the training pipeline. We will make use of 🤗's [Trainer](https://huggingface.co/transformers/master/main_classes/trainer.html?highlight=trainer) for which we essentially need to do the following:

- Define a data collator. In contrast to most NLP models, Wav2Vec2 has a much larger input length than output length. *E.g.*, a sample of input length 50000 has an output length of no more than 100. Given the large input sizes, it is much more efficient to pad the training batches dynamically meaning that all training samples should only be padded to the longest sample in their batch and not the overall longest sample. Therefore, fine-tuning Wav2Vec2 requires a special padding data collator, which we will define below

- Evaluation metric. During training, the model should be evaluated on the word error rate. We should define a `compute_metrics` function accordingly

- Load a pretrained checkpoint. We need to load a pretrained checkpoint and configure it correctly for training.

- Define the training configuration.

After having fine-tuned the model, we will correctly evaluate it on the test data and verify that it has indeed learned to correctly transcribe speech.

### Set-up Trainer

Let's start by defining the data collator. The code for the data collator was copied from [this example](https://github.com/huggingface/transformers/blob/9a06b6b11bdfc42eea08fa91d0c737d1863c99e3/examples/research_projects/wav2vec2/run_asr.py#L81).

Without going into too many details, in contrast to the common data collators, this data collator treats the `input_values` and `labels` differently and thus applies to separate padding functions on them (again making use of Wav2Vec2's context manager). This is necessary because in speech input and output are of different modalities meaning that they should not be treated by the same padding function.
Analogous to the common data collators, the padding tokens in the labels with `-100` so that those tokens are **not** taken into account when computing the loss.

In [ ]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [39]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [40]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

Next, the evaluation metric is defined. As mentioned earlier, the 
predominant metric in ASR is the word error rate (WER), hence we will use it in this notebook as well.

In [41]:
wer_metric = load_metric("wer")

Downloading:   0%|          | 0.00/1.90k [00:00<?, ?B/s]

The model will return a sequence of logit vectors:
$\mathbf{y}_1, \ldots, \mathbf{y}_m$ with $\mathbf{y}_1 = f_{\theta}(x_1, \ldots, x_n)[0]$ and $n >> m$.

A logit vector $\mathbf{y}_1$ contains the log-odds for each word in the vocabulary we defined earlier, thus $\text{len}(\mathbf{y}_i) =$ `config.vocab_size`. We are interested in the most likely prediction of the model and thus take the `argmax(...)` of the logits. Also, we transform the encoded labels back to the original string by replacing `-100` with the `pad_token_id` and decoding the ids while making sure that consecutive tokens are **not** grouped to the same token in CTC style ${}^1$.

In [ ]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [42]:
import random


def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)
    
    if isinstance(label_str, list):
        if isinstance(pred_str, list) and len(pred_str) == len(label_str):
            for index in random.sample(range(len(label_str)), 3):
                print(f'reference: "{label_str[index]}"')
                print(f'predicted: "{pred_str[index]}"')

        else:
            for index in random.sample(range(len(label_str)), 3):
                print(f'reference: "{label_str[index]}"')
                print(f'predicted: "{pred_str}"')

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

Now, we can load the pretrained `Wav2Vec2` checkpoint. The tokenizer's `pad_token_id` must be to define the model's `pad_token_id` or in the case of `Wav2Vec2ForCTC` also CTC's *blank token* ${}^2$. To save GPU memory, we enable PyTorch's [gradient checkpointing](https://pytorch.org/docs/stable/checkpoint.html) and also set the loss reduction to "*mean*".

In [ ]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-large-xlsr-53",
    # ctc_loss_reduction="mean", 
    # pad_token_id=processor.tokenizer.pad_token_id,
    # model_name_or_path if not last_checkpoint else last_checkpoint, 
    attention_dropout=0.05,
    # hidden_dropout=0.1,
    # feat_proj_dropout=0.0,
    mask_time_prob=0.075,
    layerdrop=0.1,
    gradient_checkpointing=True, 
    ctc_loss_reduction="mean", 
    ctc_zero_infinity=True,
    # bos_token_id=processor.tokenizer.bos_token_id,
    # eos_token_id=processor.tokenizer.eos_token_id,
    pad_token_id=processor.tokenizer.pad_token_id,
    # vocab_size=len(processor.tokenizer.get_vocab())
)

https://huggingface.co/facebook/wav2vec2-large-xlsr-53/resolve/main/config.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpksc1wv0u


Downloading:   0%|          | 0.00/1.73k [00:00<?, ?B/s]

storing https://huggingface.co/facebook/wav2vec2-large-xlsr-53/resolve/main/config.json in cache at /root/.cache/huggingface/transformers/8508c73cd595eb416a1d517b90762416c0bc6cfbef529578079aeae4d8c14336.7581ed2ee0c677f1e933180df51bd1a668c4a2b6d5fd1297d32069373dac097c
creating metadata file for /root/.cache/huggingface/transformers/8508c73cd595eb416a1d517b90762416c0bc6cfbef529578079aeae4d8c14336.7581ed2ee0c677f1e933180df51bd1a668c4a2b6d5fd1297d32069373dac097c
loading configuration file https://huggingface.co/facebook/wav2vec2-large-xlsr-53/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/8508c73cd595eb416a1d517b90762416c0bc6cfbef529578079aeae4d8c14336.7581ed2ee0c677f1e933180df51bd1a668c4a2b6d5fd1297d32069373dac097c
Model config Wav2Vec2Config {
  "activation_dropout": 0.0,
  "adapter_kernel_size": 3,
  "adapter_stride": 2,
  "add_adapter": false,
  "apply_spec_augment": true,
  "architectures": [
    "Wav2Vec2ForPreTraining"
  ],
  "attention_dropout": 0.05,


Downloading:   0%|          | 0.00/1.18G [00:00<?, ?B/s]

storing https://huggingface.co/facebook/wav2vec2-large-xlsr-53/resolve/main/pytorch_model.bin in cache at /root/.cache/huggingface/transformers/5d2a20b45a1689a376ec4a6282b9d9be42f931cdf8daf07c3668ba1070a059d9.622b46163a38532eae8ac5423b0481dfc0b9ea401af488b5141772bdff889079
creating metadata file for /root/.cache/huggingface/transformers/5d2a20b45a1689a376ec4a6282b9d9be42f931cdf8daf07c3668ba1070a059d9.622b46163a38532eae8ac5423b0481dfc0b9ea401af488b5141772bdff889079
loading weights file https://huggingface.co/facebook/wav2vec2-large-xlsr-53/resolve/main/pytorch_model.bin from cache at /root/.cache/huggingface/transformers/5d2a20b45a1689a376ec4a6282b9d9be42f931cdf8daf07c3668ba1070a059d9.622b46163a38532eae8ac5423b0481dfc0b9ea401af488b5141772bdff889079
Some weights of the model checkpoint at facebook/wav2vec2-large-xlsr-53 were not used when initializing Wav2Vec2ForCTC: ['project_q.bias', 'quantizer.weight_proj.weight', 'project_q.weight', 'quantizer.weight_proj.bias', 'project_hid.bias', '

In [44]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    #"facebook/wav2vec2-large-xlsr-53" if not last_checkpoint else last_checkpoint, 
    "facebook/wav2vec2-large-robust",
    attention_dropout=0.05,
    activation_dropout=0.1,
    hidden_dropout=0.1,
    feat_proj_dropout=0.01249,
    final_dropout=0.0,
    mask_time_prob=0.075,
    mask_time_length=10,
    mask_feature_prob=0,
    mask_feature_length=10,
    layerdrop=0.01377,
    gradient_checkpointing=True, 
    ctc_loss_reduction="mean", 
    ctc_zero_infinity=True,
    bos_token_id=processor.tokenizer.bos_token_id,
    eos_token_id=processor.tokenizer.eos_token_id,
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer.get_vocab())
)

Downloading:   0%|          | 0.00/1.18G [00:00<?, ?B/s]

Some weights of the model checkpoint at facebook/wav2vec2-large-robust were not used when initializing Wav2Vec2ForCTC: ['quantizer.weight_proj.bias', 'project_q.weight', 'quantizer.weight_proj.weight', 'quantizer.codevectors', 'project_hid.weight', 'project_hid.bias', 'project_q.bias']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-robust and are newly initialized: ['lm_head.bias', 'lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use

The first component of Wav2Vec2 consists of a stack of CNN layers that are used to extract acoustically meaningful - but contextually independent - features from the raw speech signal. This part of the model has already been sufficiently trained during pretrainind and as stated in the [paper](https://arxiv.org/abs/2006.11477) does not need to be fine-tuned anymore. 
Thus, we can set the `requires_grad` to `False` for all parameters of the *feature extraction* part.

In [45]:
model.freeze_feature_encoder()

In a final step, we define all parameters related to training. 
To give more explanation on some of the parameters:
- `group_by_length` makes training more efficient by grouping training samples of similar input length into one batch. This can significantly speed up training time by heavily reducing the overall number of useless padding tokens that are passed through the model
- `learning_rate` and `weight_decay` were heuristically tuned until fine-tuning has become stable. Note that those parameters strongly depend on the Timit dataset and might be suboptimal for other speech datasets.

For more explanations on other parameters, one can take a look at the [docs](https://huggingface.co/transformers/master/main_classes/trainer.html?highlight=trainer#trainingarguments).

During training, a checkpoint will be uploaded asynchronously to the hub every 400 training steps. It allows you to also play around with the demo widget even while your model is still training.

**Note**: If one does not want to upload the model checkpoints to the hub, simply set `push_to_hub=False`.

In [46]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir=save_dir,
  group_by_length=True,
  per_device_train_batch_size=8,
  per_device_eval_batch_size=8,
  gradient_accumulation_steps = 2,
  evaluation_strategy="steps",
  num_train_epochs=6,
  fp16=True,
  gradient_checkpointing=True,
  save_steps=100,
  eval_steps=100,
  logging_steps=100,
  learning_rate=1e-4,
  weight_decay=0.005,
  warmup_steps=100,
  save_total_limit=2,
)
# output_dir=save_dir,
#     group_by_length=True,
#     per_device_train_batch_size=10,
#     per_device_eval_batch_size=10,
#     gradient_accumulation_steps=2,
#     evaluation_strategy="steps",
#     num_train_epochs=0.5,
#     fp16=True,
#     save_steps=10,
#     eval_steps=10,
#     logging_steps=10,
#     learning_rate=1e-4,
#     warmup_steps=500,
#     save_total_limit=2,

Now, all instances can be passed to Trainer and we are ready to start training!

In [51]:
from transformers.utils.dummy_pt_objects import ProphetNetDecoder
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=_atcosim_train,
    eval_dataset=_atcosim_test,
    tokenizer=processor.feature_extractor,
)
# print(model)
# print(data_collator)
# print(training_args)
# print(compute_metrics)
# print(atcosim_train)
# print(processor.feature_extractor)


Using amp half precision backend




---

${}^1$ To allow models to become independent of the speaker rate, in CTC, consecutive tokens that are identical are simply grouped as a single token. However, the encoded labels should not be grouped when decoding since they don't correspond to the predicted tokens of the model, which is why the `group_tokens=False` parameter has to be passed. If we wouldn't pass this parameter a word like `"hello"` would incorrectly be encoded, and decoded as `"helo"`.

${}^2$ The blank token allows the model to predict a word, such as `"hello"` by forcing it to insert the blank token between the two l's. A CTC-conform prediction of `"hello"` of our model would be `[PAD] [PAD] "h" "e" "e" "l" "l" [PAD] "l" "o" "o" [PAD]`.

### Training

Training will take between 90 and 270 minutes depending on the GPU allocated to this notebook. While the trained model yields satisfying results on *Timit*'s test data, it is by no means an optimally fine-tuned model. The purpose of this notebook is to demonstrate how Wav2Vec2's [base](https://huggingface.co/facebook/wav2vec2-base), [large](https://huggingface.co/facebook/wav2vec2-large), and [large-lv60](https://huggingface.co/facebook/wav2vec2-large-lv60) checkpoints can be fine-tuned on any English dataset.

In case you want to use this google colab to fine-tune your model, you should make sure that your training doesn't stop due to inactivity. A simple hack to prevent this is to paste the following code into the console of this tab (*right mouse click -> inspect -> Console tab and insert code*).

```javascript
function ConnectButton(){
    console.log("Connect pushed"); 
    document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click() 
}
setInterval(ConnectButton,60000);
```

Depending on what GPU was allocated to your google colab it might be possible that you are seeing an `"out-of-memory"` error here. In this case, it's probably best to reduce `per_device_train_batch_size` to 16 or even less and eventually make use of [`gradient_accumulation`](https://huggingface.co/transformers/master/main_classes/trainer.html#trainingarguments).

In [52]:
print("step1")
train_result = trainer.train()
print("step2")

metrics = train_result.metrics
print("step3")
max_train_samples = len(_atcosim_train)
metrics["train_samples"] = min(max_train_samples, len(_atcosim_train))
print("step4")
trainer.save_model()
print("model created!")
trainer.log_metrics("train", metrics)
trainer.save_metrics("train", metrics)
trainer.save_state()

step1


/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 2970
  Num Epochs = 6
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 2
  Total optimization steps = 1116


Step,Training Loss,Validation Loss,Wer
100,6.110300,3.163630,1.000000
200,3.040000,3.068065,1.000000
300,2.924700,2.986059,1.000000
400,2.870800,2.838753,1.000000
500,2.611900,2.200194,0.985380
600,1.817800,1.209360,0.769006
700,1.198600,0.760623,0.409357
800,0.869400,0.554685,0.309942
900,0.680700,0.471959,0.295322
1000,0.609200,0.439984,0.283626


***** Running Evaluation *****
  Num examples = 30
  Batch size = 8


reference: "bee med one contact rhein one three two decimal four"
predicted: ""
reference: "scandinavian six one seven report your heading"
predicted: ""
reference: "~k ~l ~m three four six stand by a second"
predicted: ""


...

reference: "swissair nine three five two call rhein on one two seven three seven good day"
predicted: "swisair nine three five two cal rhein on one two seven three seven good dag"
reference: "speedbird one two four contact rhein one two seven three seven good bye"
predicted: "sbeedbir one two four contact rhein one two seven three seven good bie"
reference: "roger aerovic one zero six one for separation turn right heading one nine five"
predicted: "roger irvic one zero six one four sebarration turn right heading one nine five"


Saving model checkpoint to /content/drive/MyDrive/wav2vec2-robust-atcosim/checkpoint-1100
Configuration saved in /content/drive/MyDrive/wav2vec2-robust-atcosim/checkpoint-1100/config.json
Model weights saved in /content/drive/MyDrive/wav2vec2-robust-atcosim/checkpoint-1100/pytorch_model.bin
Feature extractor saved in /content/drive/MyDrive/wav2vec2-robust-atcosim/checkpoint-1100/preprocessor_config.json
Deleting older checkpoint [/content/drive/MyDrive/wav2vec2-robust-atcosim/checkpoint-900] due to args.save_total_limit


Training completed. Do not forget to share your model on huggingface.co/models =)


Saving model checkpoint to /content/drive/MyDrive/wav2vec2-robust-atcosim
Configuration saved in /content/drive/MyDrive/wav2vec2-robust-atcosim/config.json


step2
step3
step4


Model weights saved in /content/drive/MyDrive/wav2vec2-robust-atcosim/pytorch_model.bin
Feature extractor saved in /content/drive/MyDrive/wav2vec2-robust-atcosim/preprocessor_config.json


model created!
***** train metrics *****
  epoch                    =          6.0
  total_flos               = 2011389686GF
  train_loss               =       2.1036
  train_runtime            =   0:34:51.66
  train_samples            =         2970
  train_samples_per_second =         8.52
  train_steps_per_second   =        0.534


The final WER should be around 0.3 which is reasonable given that state-of-the-art phoneme error rates (PER) are just below 0.1 (see [leaderboard](https://paperswithcode.com/sota/speech-recognition-on-timit)) and that WER is usually worse than PER.

You can now upload the result of the training to the Hub, just execute this instruction:

In [ ]:
# trainer.push_to_hub()

You can now share this model with all your friends, family, favorite pets: they can all load it with the identifier "your-username/the-name-you-picked" so for instance:

```python
from transformers import AutoModelForCTC, Wav2Vec2Processor

model = AutoModelForCTC.from_pretrained("patrickvonplaten/wav2vec2-base-timit-demo-google-colab")
processor = Wav2Vec2Processor.from_pretrained("patrickvonplaten/wav2vec2-base-timit-demo-google-colab")
```

### Evaluate

In the final part, we run our model on some of the validation data to get a feeling for how well it works.

Let's load the `processor` and `model`.

In [ ]:
processor = Wav2Vec2Processor.from_pretrained("patrickvonplaten/wav2vec2-base-timit-demo-google-colab")

https://huggingface.co/patrickvonplaten/wav2vec2-base-timit-demo-google-colab/resolve/main/preprocessor_config.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpw92kh8o2


Downloading:   0%|          | 0.00/215 [00:00<?, ?B/s]

storing https://huggingface.co/patrickvonplaten/wav2vec2-base-timit-demo-google-colab/resolve/main/preprocessor_config.json in cache at /root/.cache/huggingface/transformers/17fa07f6e2f88edf24c71037bed1b7cf0cf9acfef7d3353afb589689627542c1.0e3e6656f99a6f7b9eddd943463eb7f34363640fce9e87b047ebd50d4b112b50
creating metadata file for /root/.cache/huggingface/transformers/17fa07f6e2f88edf24c71037bed1b7cf0cf9acfef7d3353afb589689627542c1.0e3e6656f99a6f7b9eddd943463eb7f34363640fce9e87b047ebd50d4b112b50
loading feature extractor configuration file https://huggingface.co/patrickvonplaten/wav2vec2-base-timit-demo-google-colab/resolve/main/preprocessor_config.json from cache at /root/.cache/huggingface/transformers/17fa07f6e2f88edf24c71037bed1b7cf0cf9acfef7d3353afb589689627542c1.0e3e6656f99a6f7b9eddd943463eb7f34363640fce9e87b047ebd50d4b112b50
Feature extractor Wav2Vec2FeatureExtractor {
  "do_normalize": true,
  "feature_extractor_type": "Wav2Vec2FeatureExtractor",
  "feature_size": 1,
  "padding_s

Downloading:   0%|          | 0.00/217 [00:00<?, ?B/s]

storing https://huggingface.co/patrickvonplaten/wav2vec2-base-timit-demo-google-colab/resolve/main/tokenizer_config.json in cache at /root/.cache/huggingface/transformers/7bf2e07c1c7ed7e3d055d87aa6bd33c75e6883d98029ae3744fed6bea9e35095.59710b1a6a5501d31e746b6e464f5c44de3e55a58f80634196025936683a68a9
creating metadata file for /root/.cache/huggingface/transformers/7bf2e07c1c7ed7e3d055d87aa6bd33c75e6883d98029ae3744fed6bea9e35095.59710b1a6a5501d31e746b6e464f5c44de3e55a58f80634196025936683a68a9
https://huggingface.co/patrickvonplaten/wav2vec2-base-timit-demo-google-colab/resolve/main/vocab.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpc5h2wq1n


Downloading:   0%|          | 0.00/268 [00:00<?, ?B/s]

storing https://huggingface.co/patrickvonplaten/wav2vec2-base-timit-demo-google-colab/resolve/main/vocab.json in cache at /root/.cache/huggingface/transformers/f739813380b0880ba302557f0f990f0f64e8efe81fcc91a4fce08206a91ee215.c44a036121683e19f1a033a1765d4344364cb4633a6cbf5cf51b153f2388abb1
creating metadata file for /root/.cache/huggingface/transformers/f739813380b0880ba302557f0f990f0f64e8efe81fcc91a4fce08206a91ee215.c44a036121683e19f1a033a1765d4344364cb4633a6cbf5cf51b153f2388abb1
https://huggingface.co/patrickvonplaten/wav2vec2-base-timit-demo-google-colab/resolve/main/special_tokens_map.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpekwb2jco


Downloading:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

storing https://huggingface.co/patrickvonplaten/wav2vec2-base-timit-demo-google-colab/resolve/main/special_tokens_map.json in cache at /root/.cache/huggingface/transformers/c239f3738252f8d8e83d9813ea82866aeec476c467a008db86229f4edc6490b0.a21d51735cf8667bcd610f057e88548d5d6a381401f6b4501a8bc6c1a9dc8498
creating metadata file for /root/.cache/huggingface/transformers/c239f3738252f8d8e83d9813ea82866aeec476c467a008db86229f4edc6490b0.a21d51735cf8667bcd610f057e88548d5d6a381401f6b4501a8bc6c1a9dc8498
loading file https://huggingface.co/patrickvonplaten/wav2vec2-base-timit-demo-google-colab/resolve/main/vocab.json from cache at /root/.cache/huggingface/transformers/f739813380b0880ba302557f0f990f0f64e8efe81fcc91a4fce08206a91ee215.c44a036121683e19f1a033a1765d4344364cb4633a6cbf5cf51b153f2388abb1
loading file https://huggingface.co/patrickvonplaten/wav2vec2-base-timit-demo-google-colab/resolve/main/tokenizer_config.json from cache at /root/.cache/huggingface/transformers/7bf2e07c1c7ed7e3d055d87aa6b

In [ ]:
model = Wav2Vec2ForCTC.from_pretrained("patrickvonplaten/wav2vec2-base-timit-demo-google-colab").cuda()

https://huggingface.co/patrickvonplaten/wav2vec2-base-timit-demo-google-colab/resolve/main/config.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmp_wdyrl76


Downloading:   0%|          | 0.00/2.29k [00:00<?, ?B/s]

storing https://huggingface.co/patrickvonplaten/wav2vec2-base-timit-demo-google-colab/resolve/main/config.json in cache at /root/.cache/huggingface/transformers/7606a293c66b1cba6bb9a342dcffe3831dc19a120cfeeb9e57cdf6ffb9630f3f.3202909be16e9c482257d0c4d35b4f4c2558cea585027c729eed803ba4bf0d8e
creating metadata file for /root/.cache/huggingface/transformers/7606a293c66b1cba6bb9a342dcffe3831dc19a120cfeeb9e57cdf6ffb9630f3f.3202909be16e9c482257d0c4d35b4f4c2558cea585027c729eed803ba4bf0d8e
loading configuration file https://huggingface.co/patrickvonplaten/wav2vec2-base-timit-demo-google-colab/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/7606a293c66b1cba6bb9a342dcffe3831dc19a120cfeeb9e57cdf6ffb9630f3f.3202909be16e9c482257d0c4d35b4f4c2558cea585027c729eed803ba4bf0d8e
Model config Wav2Vec2Config {
  "_name_or_path": "facebook/wav2vec2-base",
  "activation_dropout": 0.0,
  "adapter_kernel_size": 3,
  "adapter_stride": 2,
  "add_adapter": false,
  "apply_spec_augment":

Downloading:   0%|          | 0.00/360M [00:00<?, ?B/s]

storing https://huggingface.co/patrickvonplaten/wav2vec2-base-timit-demo-google-colab/resolve/main/pytorch_model.bin in cache at /root/.cache/huggingface/transformers/666767bce47a87d50dfd1ceac759d5dc9db6eac8d185ae8f9a7dd388687c28c5.20bdf845b9896c1d0a213f03187b544f2a12b23d751b88c7b49a95d00bd946e1
creating metadata file for /root/.cache/huggingface/transformers/666767bce47a87d50dfd1ceac759d5dc9db6eac8d185ae8f9a7dd388687c28c5.20bdf845b9896c1d0a213f03187b544f2a12b23d751b88c7b49a95d00bd946e1
loading weights file https://huggingface.co/patrickvonplaten/wav2vec2-base-timit-demo-google-colab/resolve/main/pytorch_model.bin from cache at /root/.cache/huggingface/transformers/666767bce47a87d50dfd1ceac759d5dc9db6eac8d185ae8f9a7dd388687c28c5.20bdf845b9896c1d0a213f03187b544f2a12b23d751b88c7b49a95d00bd946e1
All model checkpoint weights were used when initializing Wav2Vec2ForCTC.

All the weights of Wav2Vec2ForCTC were initialized from the model checkpoint at patrickvonplaten/wav2vec2-base-timit-demo-

Now, we will make use of the `map(...)` function to predict the transcription of every test sample and to save the prediction in the dataset itself. We will call the resulting dictionary `"results"`. 

**Note**: we evaluate the test data set with `batch_size=1` on purpose due to this [issue](https://github.com/pytorch/fairseq/issues/3227). Since padded inputs don't yield the exact same output as non-padded inputs, a better WER can be achieved by not padding the input at all.

In [61]:
input_dict = processor(_atcosim_test["input_values"][2], return_tensors="pt", padding=True)

logits = model(input_dict.input_values.to("cuda")).logits

pred_ids = torch.argmax(logits, dim=-1)[0]

It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


In [54]:
atcosim_test_transcription = load_dataset("csv", data_files={"test": "/content/sample_data/ATC_data/train_test_data/test.csv"}, delimiter="\t")["test"]

  0%|          | 0/1 [00:00<?, ?it/s]

In [62]:
print("Prediction:")
print(processor.decode(pred_ids))

print("\nReference:")
print(atcosim_test_transcription["transcription"][2].lower())

Prediction:
alittalia four eight seven contact zuich on one three four six

Reference:
alitalia four eight seven contact zurich on one three four six 


In [63]:
def map_to_result(batch):
  with torch.no_grad():
    input_values = torch.tensor(batch["input_values"], device="cuda").unsqueeze(0)
    logits = model(input_values).logits

  pred_ids = torch.argmax(logits, dim=-1)
  batch["pred_str"] = processor.batch_decode(pred_ids)[0]
  batch["text"] = processor.decode(batch["labels"], group_tokens=False)
  
  return batch

In [64]:
results = _atcosim_test.map(map_to_result, remove_columns=_atcosim_test.column_names)

0ex [00:00, ?ex/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/checkpoint.py:25: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")


Let's compute the overall WER now.

In [65]:
print("Test WER: {:.3f}".format(wer_metric.compute(predictions=results["pred_str"], references=results["text"])))

Test WER: 0.354


21.8% WER - not bad! Our demo model would have probably made it on the official [leaderboard](https://paperswithcode.com/sota/speech-recognition-on-timit).

Let's take a look at some predictions to see what errors are made by the model.

In [66]:
show_random_elements(results)

,pred_str,text
0,lufthansa for four zero eight contact not zurich one three four decimal six,lufthansa four four zero eight contact now zurich one three four decimal six
1,aasbeed riehe two zero eight is identified huever contact rhein one three two decimal four,ah speedway two zero eight is identified however contact rhein one three two decimal four
2,gl m threee four six tdembo a seccand,~k ~l ~m three four six stand by a second
3,soalair two five five seven cal rhein on one three two four god da,sobelair two five five seven call rhein on one three two four good day
4,lufthansa three six zero four contine clim to flight level three three zero,lufthansa three six zero four continue climb to flight level three three zero
5,rshd,yes go ahead
6,swissair nine three five two cal rhein on one two seven three seven god a,swissair nine three five two call rhein on one two seven three seven good day
7,erovic one zero six one roceedi direct to thransadin exbect t desen ind to the suees amarian bot to minats,aerovic one zero six one proceed direct to trasadingen expect a descent into the ~c ~v ~s ~m area in about two minutes
8,lufthansa for two one nine iss identified our clird direct to delta gilo bravao,lufthansa four two one nine is identified you're cleared direct to delta kilo bravo
9,nago delta india raro ecco charlie decend to fligh leveel three six zero,fl fl delta india bravo echo charlie descend to flight level three six zero


It becomes clear that the predicted transcriptions are acoustically very similar to the target transcriptions, but often contain spelling or grammatical errors. This shouldn't be very surprising though given that we purely rely on Wav2Vec2 without making use of a language model.

Finally, to better understand how CTC works, it is worth taking a deeper look at the exact output of the model. Let's run the first test sample through the model, take the predicted ids and convert them to their corresponding tokens.

In [ ]:
model.to("cuda")

with torch.no_grad():
  logits = model(torch.tensor(timit["test"][:1]["input_values"], device="cuda")).logits

pred_ids = torch.argmax(logits, dim=-1)

# convert ids to tokens
" ".join(processor.tokenizer.convert_ids_to_tokens(pred_ids[0].tolist()))

'[PAD] [PAD] [PAD] [PAD] [PAD] t t h h e | | | b [PAD] [PAD] [PAD] u u n n g g [PAD] [PAD] l l l l [PAD] o o o | | w w a a s s | | [PAD] [PAD] [PAD] p l l [PAD] e s s s s [PAD] n n t t [PAD] l l l y y | | [PAD] s s s i i t t t [PAD] u u u u u [PAD] [PAD] [PAD] [PAD] a a t t [PAD] e e d d | | [PAD] n n e e a a r | | t t h e e | | s s h h [PAD] [PAD] [PAD] [PAD] o o r r r r | [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]'

The output should make it a bit clearer how CTC works in practice. The model is to some extent invariant to speaking rate since it has learned to either just repeat the same token in case the speech chunk to be classified still corresponds to the same token. This makes CTC a very powerful algorithm for speech recognition since the speech file's transcription is often very much independent of its length.

I again advise the reader to take a look at [this](https://distill.pub/2017/ctc) very nice blog post to better understand CTC.